# Analysis of the predominant phase

$P_a$ = the expected proportion of A-units activated by a stimulus of a given size<br>
$P_c$ = the conditional probability that an A-unit which responds to a given stimulus ($S_1$), will allso respond to another given stimulus ($S_2$)<br><br>
It can be shown that as the size of the retina is increased,<br>
the number of S-points($N_s$) quickly ceases to be a significant parameter,<br>
and the values of $P_a$ and $P_c$ approach the value that they would have for a retina with infinitely many points.<br><br>
For a large retina, therefore, the equations are as follows:<br><br>($P_a$ approach)<br>
$$
P_a = \sum^{x}_{e=\theta} \sum^{min(y, e-\theta)}_{i=\theta} P(e,i)
$$

$$
where
$$

$$
P(e,i) = 
\begin{equation}
    \begin{pmatrix}
    x \\
    e\\
    \end{pmatrix}
R^{e}(1-R)^{x-e} \times 
    \begin{pmatrix}
    y \\
    i\\
    \end{pmatrix}
\end{equation}
R^{i}(1-R)^{y-i}
$$
and<br>
$R$ = proportion of S-points activated by the stimulus<br>
$x$ = number of excitatory connections to each A-unit<br>
$y$ = number of inhibitory connections to each A-unit<br>
$\theta$ = threshold of A-units<br>
(The quantities $e$ and $i$ are the excitatory and inhibitory components of the excitation received by the A-unit from the stimulus.<br>
If the algebraic sum $\alpha = e+i$ is equal to or greater than $\theta$, the A-unit is assumed to respond)

In [1]:
import numpy as np
from scipy.special import comb

def calculate_Pa(R,x,y,theta):
    def P(e, i):
        term1 = comb(x, e) * (R**e) * ((1-R)**(x-e))
        term2 = comb(y, i) * (R**i) * ((1-R)**(y-i))
        return term1 * term2
    Pa = sum(sum(P(e,i) for i in range(max(theta, e - theta), min(y, e - theta)+1)) for e in range(theta, x+1))
    return Pa

($P_{c}$ approach)
$$
P_{c} = \frac{1}{P_{a}} \sum^{x}_{e= \theta } \sum^{y}_{i={e-\theta}} \sum^{e}_{l_{e}=0} \sum^{i}_{l_{i}=0} \sum^{x-e}_{g_{e}=0} \sum^{y-i}_{g_{i}=0} P(e,i,l_{e}, l_{i}, g_{e}, g_{i})
$$
$$
(e - i - l_{e} + l_{i} + g_{e} - g_i \ge \theta)
$$
$$
where
$$
$$
\begin{equation}
\begin{aligned}
    &P(e,i,l_{e}, l_{i}, g_{e}, g_{i})\\
    &=\binom{x}{e} R^{e}(1 - R)^{x - e} \\
    &\times\binom{y}{i} R^{i}(1 - R)^{y - i} \\
    &\times \binom{e}{l_{e}} L^{l_{e}}(1-L)^{e - l_{e}} \\
    &\times \binom{i}{l_{i}} L^{l_{i}}(1-L)^{i - l_{i}} \\
    &\times \binom{x - e}{g_{e}} G^{g_{e}}(1-G)^{x - e - g_{e}} \\
    &\times \binom{y - i}{g_{i}} G^{g_{i}}(1-G)^{y - i - g_{i}} \\
\end{aligned}
\end{equation}
$$
and<br>
$L$ = proportion of the S-points illuminated by the first stimulus, $S_1$, which are not illumintated by $S_2$<br>
$G$ = proportion of the residual S-set (left over from the first stimulus) which is included in the second stimulus($S_2$)
$$$$

In [9]:
from tqdm import tqdm

def calculate_Pa(R,x,y,theta):
    def P(e, i):
        term1 = comb(x, e) * (R**e) * ((1-R)**(x-e))
        term2 = comb(y, i) * (R**i) * ((1-R)**(y-i))
        return term1 * term2
    Pa = sum(sum(P(e,i) for i in range(max(theta, e - theta), min(y, e - theta)+1)) for e in range(theta, x+1))
    return Pa
    
def calculate_Pc(x, y, theta, R, L, G, Pa):
    def P(e,i,le, li, ge, gi):
        term1 = comb(x, e) * (R**e) * ((1 - R)**(x - e))
        term2 = comb(y, i) * (R**i) * ((1 - R)**(y - i))
        term3 = comb(e, le) * (L**le) * ((1 - L)**(e - le))
        term4 = comb(i, li) * (L**li) * ((1 - L)**(i - li))
        term5 = comb(x - e, ge) * (G**ge) * ((1 - G)**(x - e - ge))
        term6 = comb(y - i, gi) * (G**gi) * ((1 - G)**(y - i - gi))
        return term1 * term2 * term3 * term4 * term5 * term6
    
    Pa = calculate_Pa(R, x, y, theta)
    Pc = 0
    for e in tqdm(range(theta, x + 1)):
        for i in range(e - theta, y + 1):
            for le in range(0, e + 1):
                for li in range(0, i + 1):
                    for ge in range(0, x - e + 1):
                        for gi in range(0, y - i + 1):
                            if (e - i - le + li + ge - gi) >= theta:
                                Pc+= P(e,i,le, li, ge, gi)
                                
    return Pc/ Pa if Pa != 0 else 0

In [10]:
# Example numbers

x = 10   # number of excitatory connections
y = 10   # number of inhibitory connections
theta = 3  # threshold
R = 0.5  # proportion of S-points activated by the stimulus
L = 0.3  # proportion for L
G = 0.4  # proportion for G

Pa = 0.6  # Example value for Pa, to be computed separately as per your model

Pc = calculate_Pc(x, y, theta, R, L, G, Pa)
Pc

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 39.44it/s]


1.8039399712510817

The minimum value of $P_c$ is equal to 
$$
P_{c_{min}} = (1 - L)^{x}(1 - G)^{y}
$$

In [11]:
def Pc_min(L, x, G, y):
    term1 = (1 - L)**x
    term2 = (1 - G)**y
    return term1 * term2